In [2]:
using Crystalline, MPBUtils, JLD2, LinearAlgebra, StaticArrays, SymmetryBases, PyPlot, DelimitedFiles, PrettyTables
using Crystalline: symvec2string

In [5]:
include("./symeigs/corner_charges.jl")

polarization_sg12 (generic function with 4 methods)

In [32]:
# spacegroup 2
sg = 2
id = 1
polarization = "tm"
brs = bandreps(sg, 2)
band_groupingsv = Vector{BandSummary}[]
for id in 1:4
    symeigsd, lgd = read_symdata("dim2-sg$sg-$id-res64-$polarization", dir="./symeigs/shiftedlattices/output");
    lgirsd = pick_lgirreps(lgd)
    fixup_gamma_symmetry!(symeigsd, lgd, Symbol(uppercase(polarization)))
    band_groupings = analyze_symmetry_data(symeigsd, lgirsd, brs);
    push!(band_groupingsv, band_groupings)
end

In [33]:
for (i, j, k, l) in zip(band_groupingsv...)
    base_n = i.n
    shift_0_1 = base_n[[2, 1, 3, 4, 6, 5, 7, 8, 9]]
    shift_1_0 = base_n[[1, 2, 4, 3, 6, 5, 7, 8, 9]]
    shift_1_1 = base_n[[2, 1, 4, 3, 5, 6, 7, 8, 9]]

    @assert j.n == shift_0_1
    @assert k.n == shift_1_0
    @assert l.n == shift_1_1
end

In [35]:
ws = wyckoffs(sg, 2)
for (i, j, k, l) in zip(band_groupingsv...)
    @assert corners(i, ws[4]) == corners(j)
    @assert corners(i, ws[3]) == corners(k)
    @assert corners(i, ws[2]) == corners(l)
    @assert polarizations(i, ws[4]) == polarizations(j)
    @assert polarizations(i, ws[3]) == polarizations(k)
    @assert polarizations(i, ws[2]) == polarizations(l)
end

In [37]:
# spacegroup 6
sg = 6
id = 1
polarization = "tm"
brs = bandreps(sg, 2)
band_groupingsv = Vector{BandSummary}[]
for id in 1:4
    symeigsd, lgd = read_symdata("dim2-sg$sg-$id-res64-$polarization", dir="./symeigs/shiftedlattices/output");
    lgirsd = pick_lgirreps(lgd)
    fixup_gamma_symmetry!(symeigsd, lgd, Symbol(uppercase(polarization)))
    band_groupings = analyze_symmetry_data(symeigsd, lgirsd, brs);
    push!(band_groupingsv, band_groupings)
end

In [38]:
for (i, j, k, l) in zip(band_groupingsv...)
    base_n = i.n
    shift_0_1 = base_n[[4, 3, 2, 1, 8, 7, 6, 5, 9, 10, 11, 12, 13, 14, 15, 16, 17]]
    shift_1_0 = base_n[[1, 2, 3, 4, 7, 8, 5, 6, 11, 12, 9, 10, 13, 14, 15, 16, 17]]
    shift_1_1 = base_n[[4, 3, 2, 1, 6, 5, 8, 7, 11, 12, 9, 10, 13, 14, 15, 16, 17]]

    @assert j.n == shift_0_1
    @assert k.n == shift_1_0
    @assert l.n == shift_1_1
end

In [41]:
ws = wyckoffs(sg, 2)
for (i, j, k, l) in zip(band_groupingsv...)
    @assert corners(i, ws[8]) == corners(j)
    @assert corners(i, ws[7]) == corners(k)
    @assert corners(i, ws[6]) == corners(l)
    @assert polarizations(i, ws[8]) == polarizations(j)
    @assert polarizations(i, ws[7]) == polarizations(k)
    @assert polarizations(i, ws[6]) == polarizations(l)
end

In [42]:
# spacegroup 9
sg = 9
id = 1
polarization = "tm"
brs = bandreps(sg, 2)
band_groupingsv = Vector{BandSummary}[]
for id in 1:2
    symeigsd, lgd = read_symdata("dim2-sg$sg-$id-res64-$polarization", dir="./symeigs/shiftedlattices/output");
    lgirsd = pick_lgirreps(lgd)
    fixup_gamma_symmetry!(symeigsd, lgd, Symbol(uppercase(polarization)))
    band_groupings = analyze_symmetry_data(symeigsd, lgirsd, brs);
    push!(band_groupingsv, band_groupings)
end

In [46]:
for (i, j) in zip(band_groupingsv...)
    base_n = i.n
    shift = base_n[[1, 2, 3, 4, 6, 5, 7, 8, 9, 10, 11]]
    @assert j.n == shift
end

In [50]:
ws = wyckoffs(sg, 2)
for (i, j) in zip(band_groupingsv...)
    @assert corners(i, ws[5]) == corners(j)
    @assert polarizations(i, ws[5]) == polarizations(j)
end

In [51]:
# spacegroup 10
sg = 10
id = 1
polarization = "tm"
brs = bandreps(sg, 2)
band_groupingsv = Vector{BandSummary}[]
for id in 1:2
    symeigsd, lgd = read_symdata("dim2-sg$sg-$id-res64-$polarization", dir="./symeigs/shiftedlattices/output");
    lgirsd = pick_lgirreps(lgd)
    fixup_gamma_symmetry!(symeigsd, lgd, Symbol(uppercase(polarization)))
    band_groupings = analyze_symmetry_data(symeigsd, lgirsd, brs);
    push!(band_groupingsv, band_groupings)
end

In [52]:
for (i, j) in zip(band_groupingsv...)
    base_n = i.n
    shift = base_n[[2, 1, 4, 3, 5, 6, 7, 8, 9]]
    @assert j.n == shift
end

In [56]:
ws = wyckoffs(sg, 2)
for (i, j) in zip(band_groupingsv...)
    @assert corners(i, ws[3]) == corners(j)
    @assert polarizations(i, ws[3]) == polarizations(j)
end

In [57]:
# spacegroup 11
sg = 11
id = 1
polarization = "tm"
brs = bandreps(sg, 2)
band_groupingsv = Vector{BandSummary}[]
for id in 1:2
    symeigsd, lgd = read_symdata("dim2-sg$sg-$id-res64-$polarization", dir="./symeigs/shiftedlattices/output");
    lgirsd = pick_lgirreps(lgd)
    fixup_gamma_symmetry!(symeigsd, lgd, Symbol(uppercase(polarization)))
    band_groupings = analyze_symmetry_data(symeigsd, lgirsd, brs);
    push!(band_groupingsv, band_groupings)
end

In [58]:
for (i, j) in zip(band_groupingsv...)
    base_n = i.n
    shift = base_n[[4, 3, 2, 1, 7, 8, 5, 6, 9, 10, 11, 12, 13, 14, 15]]
    @assert j.n == shift
end

In [60]:
ws = wyckoffs(sg, 2)
for (i, j) in zip(band_groupingsv...)
    @assert corners(i, ws[6]) == corners(j)
    @assert polarizations(i, ws[6]) == polarizations(j)
end

In [61]:
# spacegroup 12
sg = 12
id = 1
polarization = "tm"
brs = bandreps(sg, 2)
band_groupingsv = Vector{BandSummary}[]
for id in 1:2
    symeigsd, lgd = read_symdata("dim2-sg$sg-$id-res64-$polarization", dir="./symeigs/shiftedlattices/output");
    lgirsd = pick_lgirreps(lgd)
    fixup_gamma_symmetry!(symeigsd, lgd, Symbol(uppercase(polarization)))
    band_groupings = analyze_symmetry_data(symeigsd, lgirsd, brs);
    push!(band_groupingsv, band_groupings)
end

In [62]:
for (i, j) in zip(band_groupingsv...)
    base_n = i.n
    shift = base_n
    @assert j.n == shift
end

In [64]:
# spacegroup 13
sg = 13
id = 1
polarization = "tm"
brs = bandreps(sg, 2)
band_groupingsv = Vector{BandSummary}[]
for id in 1:3
    symeigsd, lgd = read_symdata("dim2-sg$sg-$id-res64-$polarization", dir="./symeigs/shiftedlattices/output");
    lgirsd = pick_lgirreps(lgd)
    fixup_gamma_symmetry!(symeigsd, lgd, Symbol(uppercase(polarization)))
    band_groupings = analyze_symmetry_data(symeigsd, lgirsd, brs);
    push!(band_groupingsv, band_groupings)
end

In [65]:
for (i, j, k) in zip(band_groupingsv...)
    base_n = i.n
    shift_1 = base_n[[2, 3, 1, 4, 5, 8, 6, 7, 9]]
    shift_2 = base_n[[3, 1, 2, 4, 5, 7, 8, 6, 9]]

    @assert j.n == shift_1
    @assert k.n == shift_2
end

In [71]:
ws = wyckoffs(sg, 2)
for (i, j, k) in zip(band_groupingsv...)
    @assert corners(i, ws[3]) == corners(j)
    @assert corners(i, ws[2]) == corners(k)
    @assert polarizations(i, ws[3]) == polarizations(j)
    @assert polarizations(i, ws[2]) == polarizations(k)
end

In [73]:
# spacegroup 14
sg = 14
id = 1
polarization = "tm"
brs = bandreps(sg, 2)
band_groupingsv = Vector{BandSummary}[]
for id in 1:3
    symeigsd, lgd = read_symdata("dim2-sg$sg-$id-res64-$polarization", dir="./symeigs/shiftedlattices/output");
    lgirsd = pick_lgirreps(lgd)
    fixup_gamma_symmetry!(symeigsd, lgd, Symbol(uppercase(polarization)))
    band_groupings = analyze_symmetry_data(symeigsd, lgirsd, brs);
    push!(band_groupingsv, band_groupings)
end

In [74]:
for (i, j, k) in zip(band_groupingsv...)
    base_n = i.n
    shift_1 = base_n[[1, 2, 3, 4, 5, 8, 6, 7, 9]]
    shift_2 = base_n[[1, 2, 3, 4, 5, 7, 8, 6, 9]]

    @assert j.n == shift_1
    @assert k.n == shift_2
end

In [76]:
ws = wyckoffs(sg, 2)
for (i, j, k) in zip(band_groupingsv...)
    @assert corners(i, ws[4]) == corners(j)
    @assert corners(i, ws[3]) == corners(k)
    @assert polarizations(i, ws[4]) == polarizations(j)
    @assert polarizations(i, ws[3]) == polarizations(k)
end

In [86]:
#Spacegroup 15

#In spacegroup 15, we have several Wyckoff positions as candidate centers.
#However, only the (0, 0) position respects all #spacegroup 15 symmetries, while the others respect only the C3 symmetry.
#Therefore, we analyze spacegroup 15 in the #subgroup, c3. 
sg = 15
polarization = "tm"
brs_13 = bandreps(13, 2)
brs_15 = bandreps(15, 2)
band_groupingsv_13 = Vector{BandSummary}[]
band_groupingsv_15 = Vector{BandSummary}[]

for id in 2:4
    symeigsd, lgd = read_symdata("dim2-sg$sg-$id-res64-$polarization", sgnum=13, dir="./symeigs/shiftedlattices/output");
    lgirsd = pick_lgirreps(lgd)
    fixup_gamma_symmetry!(symeigsd, lgd, Symbol(uppercase(polarization)))
    band_groupings = analyze_symmetry_data(symeigsd, lgirsd, brs_13);
    push!(band_groupingsv_13, band_groupings)
end

for id in 1:1
    println(sg)
    symeigsd, lgd = read_symdata("dim2-sg$sg-$id-res64-$polarization", dir="./symeigs/shiftedlattices/output");
    lgirsd = pick_lgirreps(lgd)
    fixup_gamma_symmetry!(symeigsd, lgd, Symbol(uppercase(polarization)))
    band_groupings = analyze_symmetry_data(symeigsd, lgirsd, brs_15);
    push!(band_groupingsv_15, band_groupings)
end


15


In [87]:
for (idx, (j, k, i)) in enumerate(zip(band_groupingsv_13...))
    idx > 10 && break
    base_n = i.n
    shift_1 = base_n[[2, 3, 1, 4, 5, 8, 6, 7, 9]]
    shift_2 = base_n[[3, 1, 2, 4, 5, 7, 8, 6, 9]]

    @assert j.n == shift_1
    @assert k.n == shift_2
end

In [127]:
ws = wyckoffs(13, 2)
for (i, j, k, l) in zip(cumsum.(band_groupingsv_13)..., cumsum.(band_groupingsv_15)...)
    k.n[end] == l.n[end] || continue
    @assert corners(k) == corners(l, ws[4])
    @assert polarizations(k, ws[4]) == polarizations(l)
end
for (i, j, k, l) in zip(cumsum.(band_groupingsv_13)..., cumsum.(band_groupingsv_15)...)
    k.n[end] == l.n[end] || continue
    @assert corners(i) == corners(l, ws[3])
    @assert polarizations(i) == polarizations(l, ws[3])
end
for (i, j, k, l) in zip(cumsum.(band_groupingsv_13)..., cumsum.(band_groupingsv_15)...)
    j.n[end] == l.n[end] || continue
    @assert corners(j) == corners(l, ws[2])
    @assert polarizations(j) == polarizations(l, ws[2])
end

In [231]:
brs_15 = bandreps(15, 2)
brs_13 = bandreps(13, 2);

In [232]:
# Below, we check that the band representations in spacegroup 15 have the corner charges we'd expect

In [267]:
for wyckoff in wyckoffs(13, 2)[end-2:end]
    a  = corner_sg13(brs_13[1].irvec, brs_13[1].irlabs, wyckoff)
    b  = corner_sg13(brs_13[3].irvec, brs_13[1].irlabs, wyckoff)
    @assert corner_sg15(brs[1].irvec, brs[1].irlabs, wyckoff) == a + b
end

1c: [2/3, 1/3]   0.3333333333333333 0.0
1b: [1/3, 2/3]   0.0 0.0
1a: [0, 0]   0.0 0.3333333333333333


In [256]:
for wyckoff in wyckoffs(13, 2)[end-2:end]
    @assert corner_sg15(brs[2].irvec, brs[2].irlabs, wyckoff) == corner_sg13(brs_13[2].irvec + brs_13[4].irvec, brs_13[1].irlabs, wyckoff)
end

In [262]:
for wyckoff in wyckoffs(13, 2)[end-2:end]
    @assert corner_sg15(brs[3].irvec, brs[3].irlabs, wyckoff) == corner_sg13(brs_13[5].irvec, brs_13[1].irlabs, wyckoff)
end

In [269]:
wyckoffs(13, 2)

4-element Vector{WyckoffPosition{2}}:
 3d: [α, β]
 1c: [2/3, 1/3]
 1b: [1/3, 2/3]
 1a: [0, 0]

In [268]:
bandreps(13, 2)

BandRepSet (⋕13): 6 BandReps, sampling 8 LGIrreps (spin-1 w/ TR)
──────┬────────────────────────
      │ 1c  1c  1b  1b  1a  1a 
      │ A   E   A   E   A   E  
──────┼────────────────────────
 KA₁  │ ·   1   ·   1   1   ·  
 KA₂  │ 1   ·   ·   1   ·   1  
 KA₃  │ ·   1   1   ·   ·   1  
 Γ₁   │ 1   ·   1   ·   1   ·
 Γ₂Γ₃ │ ·   1   ·   1   ·   1
 K₁   │ ·   1   ·   1   1   ·  
 K₂   │ ·   1   1   ·   ·   1  
 K₃   │ 1   ·   ·   1   ·   1  
──────┼────────────────────────
 μ    │ 1   2   1   2   1   2  
──────┴────────────────────────
  KVecs (maximal only): KA, Γ, K